## 5.1 Image Captioning
→CV分野の中でも特に難しい分野。  
物体検知のみならず、被写体同士の関係性を考慮して、それを文章に出力する必要があるから。  
代表的なモデルはGoogleが2015年に発表した"Show and Tell"  
画像分類のために訓練されたCNNを用いて入力画像を順伝播させ、出力層の直前の特徴を抽出してRNNへの入力とし、そのRNNがキャプションを作成するというもの。  
このCNNはすでに学習済みの重みを利用しており、このように訓練されたモデルを別のタスクに利用することを転移学習と呼ぶ。  

## 5.3 大規模データセットを扱う際の注意点
まとめ  
- 大量の画像データとメタデータを整形してバイナリに固める。
- そのバイナリを逐次読み込んで別スレッドでミニバッチを作成してキューに格納するようにする。
  
今までは全てのデータセットを`placeholder`経由で入力・正解データのミニバッチの受け渡しをしてきたが、今回のように数十GBのデータセットでは全てのデータを一度にメモリにロードすることはできない。  
そのため、今までのように`placeholder`経由でやり取りをする場合には、ミニバッチはディスクからその都度データを読み込んで加工して作成する必要がある。  
しかし、そうするとこれまでは毎ステップメモリアクセスで済んでいたデータ読み込みにディスクIOが発生してしまう。  
ディスクIOはメモリアクセスの100分の1から1,000文の1程度の速度なので、これが訓練の毎ステップに発生してしまうと、どれだけハイスペックなハードを使って効率的に訓練を行ったとしても、相当な学習時間を要してしまう。  
    
これを解消するために、ディスクからデータをロードしてデータセットを生成するスレッドと訓練を行うスレッドを分割して、キューイングをうまく行うことで、訓練を行っている間にミニバッチを作成するようにするのが定石。
  
また、もう一つの工夫として、今回の例のように一つ一つは容量が小さい画像データが数十万あるようなケースだと、画像データをひとつが100MB程度になるバイナリにまとめて、それから読み込むほうが効率的。  
大量の小さい容量を逐次読み込むのは総容量が同じで1つのファイルを読み込むよりも遥かに時間がかかる。  
  
この骨が折れるような作業も、簡単かつ高速で処理できるAPIがTensorFlowには用意されている。

## 5.4 TFRecord形式によるデータのバイナリ化
本来バイナリ化は、仕様を細かく決めた上で慎重にバイト数を意識してデータのエンコード・デコードを行うことが必要だが、`TFRecord`を使えば簡単にデータの読み書きが行える。  
他の方法でもバイナリ化は可能だが、**TensorFlowは`TFRecord`を推奨しており、他の方法と比べて高速でデータの読み書きが可能である。**  
  
### Writing
`TFRecord`形式でファイルを書き込む基本的な方法として、1件のデータセットに相当する`tf.train.Example()`クラスに必要なデータを登録して、それを専用の`writer`を用いて書き込むという手順を取る。  
  
`tf.train.Features()`（複数形）はキーバリュー形式での登録を行うクラスで、`tf.train.Feature()`（単数形）は1つのバリューを定義するクラスなので注意する。  
データの型によって後者の引数が変わるので、詳しくはリファレンスか本書を参照。

In [1]:
#!/usr/bin/env python
# coding=utf-8

import glob
import tensorflow as tf


# Get a file path
IMG_LIST = [i for i in glob.glob('img/*.jpg')]

# Export as 'test.tfrecord'
with tf.python_io.TFRecordWriter('test.tfrecord') as w:
    for img in IMG_LIST:

        # 'rb' indicates to read files as binary data
        with tf.gfile.FastGFile(img, 'rb') as f:
            data = f.read()

        # Resister read bytes to key and value
        features = tf.train.Features(feature={
            'data': tf.train.Feature(bytes_list=tf.train.BytesList(
                value=[data]))
        })

        # Write into Example Class
        example = tf.train.Example(features=features)
        w.write(example.SerializeToString())

/home/tk/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Reading
`TFRecord`ファイルを読み込む方法はいくつかあり、`TFRecord`専用の`reader`も存在するが、ここではTensorFlowのDataSetAPIを使って解説を行う。  
DataSetAPIとはモデルの読み込み、加工、バッチ化、シャッフルなどの機械学習におけるデータセット周りで必須の面倒な処理を簡単に記述できる非常に強力なAPIのこと。  
  
DataSetAPIには2つの重要なクラスが存在する。  
`tf.data.Dataset()`クラスと、`tf.data.Iterator()`クラス。
  
- Dataset　・・・　データセットの集合を管理し、加工やバッチ化を行う。
- Iterator　・・・  Datasetオブジェクトをどのようにループさせるかを管理する。
  
`Dataset`クラスの`from_tesnors()`や`from_tensor_slices()`メソッドを用いればTesnorやTensorの配列からデータセットを読み込むことができる。  
今回のように`TFRecord`形式のデータを読み込む際は`tf.data.TFRecordDataset()`クラスを用いる。  
引数は`TFRecord`のファイル名一覧で、普通は全データ容量が大きく、１００Mb程度の大きさに分割してバイナルファイルを管理するので、その際には引数がファイル名の配列になる。  
`map()`メソッドを用いるとデータセット1レコードごとにどのような処理を食われるかということを記述できる。  
  
パース処理では`ｔｆ．FixedLegFeature()`を用いる。  
第一引数はパース対象のTensorのshape。  
パースしただけではまだバイト列のままなので、`tf.image.decode_jpeg()`オペレーションを用いることで、unit8型の3階テンソルにしている。  
  
データセットの用意が完了したので、あとは`Iterator` クラスを用いてどのようにループさせるかを定義する。  
`make_one_shot_iterator()`メソッドは順番にデータセットを一周するという基本的な`Iterator`。  
そして、`get_next()`メソッドを用いて1件ずつデータセットにアクセスするためのTensorを取得する。
  
**下のコードは単体ではエラーが出て動かないので注意（実装方法の確認用）**  

In [1]:
#!/usr/bin/env python
# coding=utf-8

import tensorflow as tf
from skimage import io

def parse(example):
    # Parse TFRecord
    features = tf.parse_single_example(
        example,
        features={
            'data': tf.FixedLenFeature([], dtype=tf.string)
        })

    # Decode bytes into jpeg
    img = features['data']
    img = tf.image.decode_jpeg(img)
    return img

# Read and parse TFRecord data
data_set = tf.data.TFRecordDataset(['test.tfrecord']).map(parse)

# Define iterator
iterator = dataset.make_one_hot_iterator()

# Get an element from iterator
next_element = iterator.get_next()

with tf.Session() as sess:
    # Get sigle image from dataset
    jpeg_img = sess.run(next_element)

    # Show jpeg_img with sckit-img
    io.imshow(jpeg_img)
    io.show()

/home/tk/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


NameError: name 'dataset' is not defined

## 5.5 データセット整形プログラム作成
  
流れは以下の通り。  

1. STAIR Captionsのファイルを読み込んで画像idからファイルパスとキャプション用の辞書を作成する
2. 訓練用、検証用、テスト用にデータを分割する
3. 訓練用のデータを元に単語辞書を作成する
4. 画像を読み込み`TFRecord`形式で画像とキャプションの組み合わせを1レコードとして`TFrecord`に書き込む
  
加えて、1つの`TFRecord`に全てのデータを書き込んでしまうとデータが大きくなりすぎてしまうために、データを分割して複数の`TFRecord`ファイルに書き込む。  
  
今回使用するのは、JSONファイルの中の`image`と`annotation`のみ。
`image`が各画像ファイルのファイル名や画像サイズとった情報を持っており、`annotation`がキャプション情報を保持している。  
  
### dataset.py  

In [ ]:
#-*- coding:utf-8 -*-
# dataset.py from
# https://github.com/thinkitcojp/TensorFlowDL-samples

import os
import json
import numpy as np
from collections import namedtuple, Counter
import tensorflow as tf

tf.flags.DEFINE_string("train_img_dir", "data/img/train2014/", "Training image directory.")
tf.flags.DEFINE_string("val_img_dir", "data/img/val2014/", "Validation image directory.")
tf.flags.DEFINE_string("train_captions", "data/stair_captions_v1.1_train.json", "Training caption file.")
tf.flags.DEFINE_string("val_captions", "data/stair_captions_v1.1_val.json", "Validation caption file.")
tf.flags.DEFINE_string("out_dir", "data/tfrecords/", "Output TFRecords directiory.")
tf.flags.DEFINE_integer("min_word_count", 4, "The minimum number of occurrences of each word in th training set for includion in the vocab.")
tf.flags.DEFINE_string("word_list_file", "data/dictionary.txt", "Output word list file.")

FLAGS = tf.flags.FLAGS

START_WORD = '<S>'
END_WORD = '<E>'
UNKNOWN_WORD = '<UNW>'

NUM_TRAIN_FILE = 256
NUM_VAL_FILE = 4
NUM_TEST_FILE = 8


ImageMetadata = namedtuple("ImageMetadata",["img_id", "filename"])

#画像メタデータと辞書をもとに、指定されたファイル数に分割してバイナリ（TFRecord）を作成する
def _create_datasets(name, img_meta, captions, word_to_id, num_file):

    #画像メタデータをだいたい等しく分割
    img_chunk = np.array_split(img_meta, num_file)
    counter = 0
    for i in range(1, num_file + 1):
        output_file_name = "%s-%.3d.tfrecord" % (name, i)
        output_file_path = os.path.join(FLAGS.out_dir, output_file_name)
        target_chunk = img_chunk[counter]
        #対象画像群書ごとにWriterを定義
        with tf.python_io.TFRecordWriter(output_file_path) as writer:
            for img in target_chunk:
                img_id = img[0]
                filename = img[1]
                #画像ファイルをバイト列として読み込み
                with tf.gfile.FastGFile(filename, "rb") as f:
                    data = f.read()

                #キャプションのid化
                caption = captions[int(img_id)]
                caption_ids = []
                for w in caption:
                    if w in word_to_id:
                        caption_ids.append(word_to_id[w])
                    else:
                        caption_ids.append(word_to_id[UNKNOWN_WORD])

                #固定長部分
                context = tf.train.Features(feature={
                    "img_id": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(img_id)])),
                    "data": tf.train.Feature(bytes_list=tf.train.BytesList(value=[data])),
                    })

                #可変長部分
                caption_feature = [tf.train.Feature(int64_list=tf.train.Int64List(value=[v])) for v in caption_ids]
                feature_lists = tf.train.FeatureLists(feature_list={
                    "caption":tf.train.FeatureList(feature=caption_feature)
                    })

                #TFRecordに書き込み
                sequence_example = tf.train.SequenceExample(context=context, feature_lists=feature_lists)
                writer.write(sequence_example.SerializeToString())

        counter += 1

#jsonファイルを読み込み画像のid, ファイル名, キャプションを取得する。
def _load_metadata(caption_filename, img_dir):

    #jsonファイルをロード
    with open(caption_filename, 'r') as f:
        meta_data = json.load(f)

    #画像idとファイル名を持つnamedtupleのリストを作成
    meta_list = [ImageMetadata(x['id'], os.path.join(img_dir, x['file_name'])) for x in meta_data['images']]

    #スペース区切りのcaptionを単語の配列に変換
    def _create_word_list(caption):
        tokenized_captions = [START_WORD]
        tokenized_captions.extend(caption.split())
        tokenized_captions.append(END_WORD)
        return tokenized_captions

    #{画像id => キャプションのリスト}の辞書を作成
    id_to_captions = {}
    for annotation in meta_data["annotations"]:
        img_id = annotation['image_id']
        caption = annotation['tokenized_caption']
        caption = _create_word_list(caption)
        #キャプションはいくつかあるため１つだけを採用
        id_to_captions[img_id] = caption

    print("Loaded caption metadata for %d images from %s" % (len(meta_list), caption_filename))

    return meta_list, id_to_captions


def _create_vocab(captions):

    counter = Counter()
    for c in captions:
        counter.update(c)

    print("total words:", len(counter))
    #出現回数が一定数のものだけ辞書に採用。出現回数降順でソート
    #word_countsは(単語, 出現回数)のリスト
    word_counts = [x for x in counter.items() if x[1] >= FLAGS.min_word_count]
    word_counts.sort(key=lambda x: x[1], reverse=True)
    print("Words in vocab:", len(word_counts))


    #辞書作成
    word_list = [x[0] for x in word_counts]
    #<S>と<E>のidを1,0で固定したいので、一度削除して先頭に追加する
    word_list.remove(START_WORD)
    word_list.remove(END_WORD)
    word_list.insert(0, START_WORD)
    word_list.insert(0, END_WORD)

    word_list.append(UNKNOWN_WORD)
    word_to_id = dict([(x, y) for (y, x) in enumerate(word_list)])
    id_to_word = dict([(x, y) for (x, y) in enumerate(word_list)])
    return word_to_id, id_to_word


def main(argv):

    #jsonファイルからメタデータの読み込み
    #(画像id, ファイルパス)のタプルの配列と{id=>キャプションのリスト}を取得
    train_meta, train_captions = _load_metadata(FLAGS.train_captions, FLAGS.train_img_dir)
    val_meta, val_captions = _load_metadata(FLAGS.val_captions, FLAGS.val_img_dir)

    #キャプションをマージ
    captions = {k:v for dic in [train_captions, val_captions] for k, v in dic.items()}

    #訓練データ,バリデーションデータ,テストデータに分割
    train_cutoff = int(0.85 * len(val_meta))
    val_cutoff = int(0.90 * len(val_meta))

    train_dataset = train_meta + val_meta[0:train_cutoff]
    val_dataset = val_meta[train_cutoff:val_cutoff]
    test_dataset = val_meta[val_cutoff:]


    #訓練データから辞書作成
    train_captions = []
    for meta in train_dataset:
        c = captions[meta.img_id]
        train_captions.append(c)

    word_to_id, id_to_word = _create_vocab(train_captions)


    #画像を読み込みメタデータと結合したバイナリを作成
    _create_datasets("train", train_dataset, captions, word_to_id, NUM_TRAIN_FILE)
    _create_datasets("val", val_dataset, captions, word_to_id, NUM_VAL_FILE)
    _create_datasets("test", test_dataset, captions, word_to_id, NUM_TEST_FILE)

    # 単語リスト出力
    with open(FLAGS.word_list_file, 'a') as f:
        for k, v in id_to_word.items():
            f.write(v)
            f.write('\n')


if __name__ == "__main__":
    tf.app.run()

### namedtuple
この[サイト](http://d.hatena.ne.jp/pknight/20170323/1490234933)がわかりやすい
使いこなせたらすごく便利そう

In [12]:
from collections import namedtuple

TEST = namedtuple('TEST', ['X', 'Y'])

In [13]:
test = [TEST(i, i) for i in range(3)]

test  # [TEST(X=0, Y=0), TEST(X=1, Y=1), TEST(X=2, Y=2)]

[TEST(X=0, Y=0), TEST(X=1, Y=1), TEST(X=2, Y=2)]

In [14]:
print(test[0])  # TEST(X=0, Y=0)
print(test[1])  # TEST(X=1, Y=1)

TEST(X=0, Y=0)
TEST(X=1, Y=1)


## 3.6 Transfer Learning with GoogLeNet Inception-v3
### Inception-v3
Deep Learningを利用するとどうしても、「畳み込みサイズのフィルタのサイズやユニット数はどうしたら良いのか？」など、構造をどうした方がより良いモデルになるのかという疑問がつきまとう。  
しかもその結論は「どの値にしてもそれぞれ別の重要な特徴量を抽出することができる」である。  
そこで、「いくつかの異なるパターンを試してそれを結合してしまおう」というのがInceptionモジュールの発想らしい。  
応用例：[ラーメン二郎を識別する人工知能の中身](https://qiita.com/shouta-dev/items/26cb1ace8c11c196f86e)  
  
Inceptionについての詳しい説明は省略。  
  
### Inception-v3の書き出し 
Inception-v3を利用する方法はいくつかあるが、一番手っ取り早いのは配布済みのProtocol Buffers形式（pb形式）のファイルを用いること。  
他には、TensorFlowのハイレベルAPIである`tf.contrib.slim`パッケージを用いて書かれたInception-v3のコードを読み込んで、公開済みのチェックポイントファイルをロードして用いる方法もある。  
前者の場合は現在公開されているモデルが入力を一見しか受け付けない推論に特化した形になっているため、今回は後者を採用。  
ただ、今回の学習では画像側のネットワークは固定にしてしまいたいため、モデルとチェックポイントファイルから`Variable`を`Constant`に変換してpd形式に固め直す。  
   
### create_inception_v3_pb.py

In [ ]:
# -*- coding:utf-8 -*-
# create_inception_v3_pb.py from
# https://github.com/thinkitcojp/TensorFlowDL-samples
# Requiring inception_v3.py located in the same dir to excecute this file.

import tensorflow as tf
import inception_v3 as iv3

FLAGS = tf.flags.FLAGS
tf.flags.DEFINE_string("ckpt_file", 'ckpt/inception_v3.ckpt', "Inception-v3 checkpoint file.")
tf.flags.DEFINE_string('log_dir', 'logs/', "TensorBoard log directory.")
tf.flags.DEFINE_string('output_dir', './', "Output directory.")
tf.flags.DEFINE_string('output_file', 'inception_v3.pb', "Output file name.")

#　Inception-v3を読み込み
input_img = tf.placeholder(tf.float32,[None, 299, 299, 3], name='input_image')
arg_scope = iv3.inception_v3_arg_scope()
with tf.contrib.slim.arg_scope(arg_scope):
    logits, end_points = iv3.inception_v3(inputs=input_img, is_training=False, num_classes=1001)

#　計算グラフ取得
graph = tf.get_default_graph()

#　TensorBordで確認できるように
writer = tf.summary.FileWriter(FLAGS.log_dir, graph)
writer.close()

#　pb形式で書き出し
tf.train.write_graph(graph, FLAGS.output_dir, FLAGS.output_file)
saver = tf.train.Saver()

with tf.Session() as sess:
    #チェックポイントが読み込めるか念のために確認
    saver.restore(sess, FLAGS.ckpt_file)

`arg_scope`は`tf.contrib.slim`パッケージで出てくる引数の統一スコープ。  
毎回引数に`tf.nn.relu`などを記載するのは面倒難度絵、`arg_scope`で一回宣言すれば、同じ名前の引数には同じ引数を与えてくれるという機能を提供している。  
ここでは、inception_v3.pyに記載されているデフォルトの`arg_scope`をロードしている。  
今回は再度訓練するわけではないので、引数`is_training`は`False`にする。  
`num_classes=1001`にしている理由は、チェックポインファイルのモデルが出力層のユニット数を1001にしているから。  
デフォルトの1000にしてしまうと、チェックポイントのファイルが読み込めないらしい。  
  
ここで`FileWriter`をすぐにｃｌｏｓｅしているのは、訓練のログを取るわけではなく、計算グラフが正しく載っているか確認するためだから。  
  
グラフファイルの書き出しは`tf.train.write_graph()`メソッドで行うことができる。  
最後に念の為、チェックポイントファイルがこのグラフに合致しているものなのかを`tf.train.Saver()`クラスの`restore()`メソッドでロードして確認している。  
もし全く違うファイル、バージョンなどが異なるファイルをロードした場合はエラーになるため、デバックの代わりになる。

### freeze_graph.py
freeze_graph.pyは計算グラフが記載されたpbファイルとチェックポイントファイルを読み込んで、計算グラフ上の`ｔｆ．Variable()`をチェックポイントに記載された値に書き換えた`tf.constant()`に変換して永久保存版のモデルを作成するためのツール。  
これによって出力されたpbファイルはpythonが乗っていないモバイル状やその他のデバイス上で実行できる。  
実際に訓練したモデルを実環境に動かすときには、推論にしか用いない最小限の計算グラフを構築した後に、チェックポイントとファイルと一緒にfreeze_graph.pyで固めたpbファイルを用いる。  
  
freeze_graph.pyは以下の4つの引数を取る。  
  
- `input_graph`: 入力となる計算グラフのpbファイル名
- `input_checkpoint`: 学習済みのチェックポイントファイル
- `output_node_names`: 実行用pb形式として出力したいTensorの名前
- `output_graph`: 出力pbファイル名
  
今回は`output_node_names`には、Softmax層の直前の値を指定する。
ただし、今回のようにモデルのコードを自分で書いたわけではない場合、`output_node_names`に指定する値を長々としたコードから探すのは手間。  
そういうときに、TensorBoardのGRAPHSを利用するとすぐに確認できる。  

### im2txt.py
詳細は本文を確認されたし。  

#### Teacher Forcing
学習初期はあてずっぽうな単語を生成するため、長期時系列の学習は非常に時間がかかる。  
そのため、正解がわかっている学習のときには各時刻のRNNの入力として前の時刻の正解データを入れてあげることで学習速度を加速させる。  
これをTeacher Forcingという。  
https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/  
https://satopirka.com/2018/02/encoder-decoder%E3%83%A2%E3%83%87%E3%83%AB%E3%81%A8teacher-forcingscheduled-samplingprofessor-forcing/  

In [ ]:
# -*- coding:utf-8 -*-
# im2txt.py from
# https://github.com/thinkitcojp/TensorFlowDL-samples
# This file requires PIL to run
# Install by running '$ pip install Pillow' if you don't have PIL

import glob
import tensorflow as tf
from PIL import Image
import numpy as np 


FLAGS = tf.flags.FLAGS
tf.flags.DEFINE_string('tfrecord_dir', 'data/tfrecords/', "TFRecords' directory.")
tf.flags.DEFINE_string('dictionary_path', 'data/dictionary.txt', "Dictionary file path.")
tf.flags.DEFINE_string('inference_pb', 'data/im2txt.pb', "Inference Graph pb file path.")
tf.flags.DEFINE_string('img_embedding_pb', 'create_inception_v3/inception_v3_freezed.pb',"Image embedding network pb file path.")
tf.flags.DEFINE_string('model_dir', 'ckpt/', "Saved checkpoint directory.")
tf.flags.DEFINE_string('log_dir', 'logs/', "TensorBoard log directory.")
tf.flags.DEFINE_string('test_img_dir','data/img/for_eval/' ,"Test image directory.")
tf.flags.DEFINE_integer('max_step', 100000, "Num of max train step.")
tf.flags.DEFINE_integer('eval_interval', 100, "Step interval of evaluation.")
tf.flags.DEFINE_integer('embedding_size', 512, "Num of embedded feature size.")
tf.flags.DEFINE_integer('batch_size', 64, "Num of batch size.")
tf.flags.DEFINE_float('learning_rate', 0.0005, "Learning rate.")
tf.flags.DEFINE_float('max_gradient_norm', 5.0, "Max norm of gradient.")

EOS_ID = 0
SOS_ID = 1

def _process_img(encoded_img):

    #画像のデコード [height, width, channel]の3階テンソルになる
    img = tf.image.decode_jpeg(encoded_img)
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    #サイズが299*299になるようにリサイズ、クロップ
    img = tf.image.resize_images(img, [373, 373])
    img = tf.image.central_crop(img, 0.8)
    return img

def _parse_function(sequence_expample_proto):

    #tf.train.SequenceExampleをパースするメソッド
    context, feature_lists = tf.parse_single_sequence_example(
        sequence_expample_proto,
        context_features = {
            "img_id": tf.FixedLenFeature([], dtype=tf.int64),
            "data": tf.FixedLenFeature([], dtype=tf.string),
        },
        sequence_features = {
            "caption": tf.FixedLenSequenceFeature([], dtype=tf.int64),
        })
    #パース結果を格納、画像は加工して格納
    img = _process_img(context['data'])
    caption = feature_lists['caption']

    #正解データのサイズを取得、正解データは<SOS>がないので1を引く
    lengths = tf.size(caption) - 1
    #訓練時decoderへの入力は<EOS>がない
    decoder_input = caption[:-1]
    #正解データは<SOS>がない
    correct = caption[1:]
    
    return (img, lengths, decoder_input, correct)



def build_input(pattern, repeat_count=None ,shuffle=True):

    #訓練用、バリデーション用のtfrecordsファイル名一覧を取得
    files = []
    for file_path in glob.glob(FLAGS.tfrecord_dir+pattern):
        files.append(file_path)

    #TFRecordをパースしてDatasetを作成
    dataset = tf.data.TFRecordDataset(files).map(_parse_function).repeat(count=repeat_count)

    #1000件ずつバッファを取りながらデータをシャッフル
    if shuffle:
        dataset = dataset.shuffle(1000)
    #キャプションデータ長の不均衡を0でpaddingしてミニバッチを作成
    padded_shapes = (tf.TensorShape([300,300,3]), tf.TensorShape([]), tf.TensorShape([None]), tf.TensorShape([None]))
    dataset = dataset.padded_batch(FLAGS.batch_size, padded_shapes=padded_shapes)

    #repeatで指定した回数までループするiteratorを作成
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    return next_element

def build_img_embedding(img_input, embedding_size):

    #画像が崩れていないかTensorBoardで確認
    tf.summary.image('input', img_input*256, max_outputs=10)

    #inception-v3の訓練済みモジュールを読み込み
    with tf.gfile.FastGFile(FLAGS.img_embedding_pb, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    img_embedding = tf.import_graph_def(graph_def, input_map={'input_image':img_input}, return_elements=['InceptionV3/Predictions/Reshape:0'])

    #RNNの中間層に合わせるための全結合
    img_embedding = tf.layers.dense(img_embedding[0], FLAGS.embedding_size)

    return img_embedding


def build_caption(img_embedding, vocab_size, is_train=True, decoder_input=None, decoder_lengths=None, end_token=EOS_ID):

    with tf.name_scope("captioning"):

        batch_size = tf.shape(img_embedding)[0]
        embedding_size = FLAGS.embedding_size
        #単語組み込み用の重みを定義
        word_embedding = tf.get_variable("embeddings", [vocab_size , embedding_size])

        if is_train:
            #decoder_inputからlookup
            embedded_input = tf.nn.embedding_lookup(word_embedding, decoder_input)
            #decoder側への組み込み入力、有効時間長を引数に訓練用のヘルパーを定義
            helper = tf.contrib.seq2seq.TrainingHelper(embedded_input, decoder_lengths)
        else:
            #出力層で最も確率が高いものだけを選択し次の入力にするヘルパーを定義
            helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(word_embedding, tf.fill([batch_size], SOS_ID), end_token)

        #LSTMのCellを定義
        rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(embedding_size)
        #画像組み込みベクトルを入力にしてRNNを1ステップ計算してキャプション生成の初期値獲得
        _, initial_state = rnn_cell(img_embedding, rnn_cell.zero_state(batch_size, dtype=tf.float32))

        #出力そうの挙動を定義
        projection_layer = tf.layers.Dense(vocab_size, use_bias=False)
        #デコーダー側の挙動を確定
        decoder = tf.contrib.seq2seq.BasicDecoder(rnn_cell, helper, initial_state, projection_layer)
        #Dynamicデコード
        output, final_state, lengths = tf.contrib.seq2seq.dynamic_decode(decoder, impute_finished=True, maximum_iterations=100)

    return output, lengths

#誤差計測
def build_loss(logit, correct, lengths):
    with tf.name_scope('loss'):
        #有効でない時系列部分をマスク
        max_len = tf.reduce_max(lengths)
        weight = tf.sequence_mask(lengths, max_len, logit.dtype)
        loss = tf.contrib.seq2seq.sequence_loss(logit, correct, weight)
    return loss

#訓練実施
def build_train(loss):
    with tf.name_scope('train'):
        global_step = tf.Variable(0, trainable=False)
        #勾配クリッピング
        params = tf.trainable_variables()
        gradients = tf.gradients(loss, params)
        clipped_gradients, _ = tf.clip_by_global_norm(gradients, FLAGS.max_gradient_norm)
        #訓練
        optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate)
        update = optimizer.apply_gradients(zip(clipped_gradients, params),global_step=global_step)

    return update, global_step

#ckptからのrestoreもしくは初期化
def initialize_model(saver, sess, initializer=None):

    #最新のckptを取得
    ckpt_state = tf.train.get_checkpoint_state(FLAGS.model_dir)
    if ckpt_state:
        last_model = ckpt_state.model_checkpoint_path
        saver.restore(sess,last_model)
    else:
        sess.run(initializer)

def main(argv):


    dataset_dir = FLAGS.tfrecord_dir
    embedding_size = FLAGS.embedding_size

    #辞書ファイルの読み込み
    print("Load dictionary.")
    id_to_word = []
    with open(FLAGS.dictionary_path, 'r') as f:
        for line in f:
            word = line.rstrip('\n')
            id_to_word.append(word)
    vocab_size = len(id_to_word)

    print("Start to build model.")

    train_graph = tf.Graph() #訓練用グラフ
    infer_graph = tf.Graph() #推論用グラフ

    #推論用のグラフ
    with infer_graph.as_default():
        #推論の入力はplaceholder形式
        input_img = tf.placeholder(tf.float32,[None, 299, 299, 3], name='input_img')
        infer_embedding = build_img_embedding(input_img, embedding_size)
        infer_output, _ = build_caption(infer_embedding, vocab_size, is_train=False)

        infer_saver = tf.train.Saver()


    #訓練用のグラフ
    with train_graph.as_default():
        #訓練の入力はDataSetAPIを用いたパイプ
        img , lengths, decoder_input, correct = build_input('train-*')
        #画像組み込み
        train_embedding = build_img_embedding(img, embedding_size)
        #キャプション生成
        train_output, train_lengths = build_caption(train_embedding, vocab_size, decoder_input=decoder_input, decoder_lengths=lengths)
        #誤差計算
        loss = build_loss(train_output.rnn_output, correct, lengths)
        #訓練
        update, global_step = build_train(loss)

        #生成テキストのlogging
        text_ph = tf.placeholder(tf.string, shape=(None,), name='generated')
        tf.summary.text('text_summary', text_ph)
        #ログのマージ
        summary_op = tf.summary.merge_all()
        initializer = tf.global_variables_initializer()
        train_saver = tf.train.Saver()



    print("Finish building models.")

    #グラフごとにSessionを定義
    sess_infer = tf.Session(graph=infer_graph)
    sess_train = tf.Session(graph=train_graph)

    #TensorBoardのwriterを定義
    summary_writer = tf.summary.FileWriter(FLAGS.log_dir, train_graph)

    #inferenceをpb形式でエクスポート
    tf.train.write_graph(infer_graph.as_graph_def(), './data', 'infer_graph.pb', as_text=False)

    #初期化
    initialize_model(train_saver, sess_train, initializer)

    last_step = sess_train.run(global_step)
    print("start training!")
    for i in range(FLAGS.max_step):
        step = last_step + i + 1

        sess_train.run(update)

        #一定ステップごとにモデル保存とloggingを行い文章生成を行う
        if step % FLAGS.eval_interval == 0:
            print("Step%d. Save model."%step)
            #モデル保存
            train_saver.save(sess_train, FLAGS.model_dir+'mymodel')

            #inference用のgraphにckptをロード
            print("Restore to inference graph.")
            initialize_model(infer_saver, sess_infer)

            #チェック用の画像ファイル一覧を取得
            eval_images = []
            for file_path in glob.glob(FLAGS.test_img_dir+'*.jpg'):
                eval_images.append(file_path)

            #画像をモデルの入力に変換
            inference_input = []
            for file in eval_images:
                _img = Image.open(file)
                inference_input.append(np.asarray(_img.resize((299,299)))/255.0)

            #実行
            infer_result = sess_infer.run(infer_output.sample_id, feed_dict={input_img: inference_input})
            #結果を辞書で変換
            captions = []
            for result in infer_result:
                caption = ''
                for j in result:
                    caption += id_to_word[j]
                captions.append(caption)

            #TensorBoardのlogging
            run_opt = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_meta = tf.RunMetadata()
            summary_val = sess_train.run(summary_op, feed_dict={text_ph:captions}, options=run_opt, run_metadata=run_meta)
            summary_writer.add_summary(summary_val, step)
            summary_writer.add_run_metadata(run_meta,'step%d'%step)


    summary_writer.close()
    sess_infer.close()
    sess_train.close()

if __name__ == '__main__':
    tf.app.run()